<a href="https://colab.research.google.com/github/gowun/BladderCancer_AMC/blob/master/Notebooks/classifier_check_20211001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.colab import drive

import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)

auth.authenticate_user()
drive.mount('/content/gdrive')
home_path = '/content/gdrive/My Drive/BladderCancer_AMC/'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


Mounted at /content/gdrive


In [2]:
!pip install lifelines
!pip install shap

     |████████████████████████████████| 348 kB 8.2 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=127209622c428e0ff23473c81d9751ed06f38caa7a768af4c221ba59d09df98f
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma
     |████████████████████████████████| 356 kB 8.8 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491648 sha256=6f03677a62a411a639f70ea6768562a007fa351cdc8e89d2c2c8f8db22935e65
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [3]:
!git clone https://github.com/gowun/BladderCancer_AMC.git

Cloning into 'BladderCancer_AMC'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 138 (delta 47), reused 77 (delta 22), pack-reused 0
Receiving objects: 100% (138/138), 4.50 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [4]:
from BladderCancer_AMC.ModelingTools import utils as ut
from BladderCancer_AMC.ModelingTools import clustering as cl
from BladderCancer_AMC.ModelingTools import tree_modeling as tm
from BladderCancer_AMC.ModelingTools import linear_modeling as lm
from BladderCancer_AMC.ModelingTools import figure as fe

In [6]:
data_labels = ['MDA_MVAC', 'MDA_DDMVAC', 'Meta_Datasets', 'AMC']
classifiers = ut.load_data(home_path + 'intersect_classifiers.pkl', 'pickle')
datasets = ut.load_data(f'{home_path}scaled_datasets_3mths_20211001.pkl', 'pickle')

In [7]:
def random_oversampling(idx_list, n):
  np.random.seed(1234)
  return np.random.choice(idx_list, n)

MAX_ROW = 100000
over_idx_dict = dict()
for i, d in enumerate(datasets['power']):
  over_idx_dict[data_labels[i]] = random_oversampling(list(range(len(d))), MAX_ROW)

In [ ]:
%%time
topK = 10
### Simple Random Forest for Variable Selection when the classifier inclued 10 more variables 
### And.. select the best normalizer
methods = ['power', 'standard', 'rankgauss']
sum_score = dict()
columns = dict()
for m in methods:
  sum_score[m] = 0
  columns[m] = dict()
  for cls, vars in classifiers.items():
    if len(vars) > topK:
      for i, l in enumerate(data_labels):
        X, y = datasets[m][i][vars].iloc[over_idx_dict[l]], np.array(datasets[m][i]['response'])[over_idx_dict[l]]
        sample_leaf = round(MAX_ROW / len(datasets[m][i]) * 3/2)
        result = tm.random_forest_with_performance([X, y], 50, 3, sample_leaf)
        sum_score[m] = sum_score[m] + result['performance']['AUC'] + result['performance']['PRAUC'] - abs(result['performance']['R2'])
        columns[m]['_'.join([cls, l])] = result['feature importance']['feature'].values[:topK]

In [9]:
sum_score

{'power': 78.74323322689307,
 'rankgauss': 78.7434957182976,
 'standard': 78.74404567804635}

In [10]:
#BEST_NOR = methods[np.argmax(list(sum_score.values()))]
#print(BEST_NOR)
BEST_NOR = 'standard'
final_columns = {}
for cls, vars in classifiers.items():
  if len(vars) > 10:
    names = list(filter(lambda x: x.startswith(cls), columns[BEST_NOR].keys()))
    tmp = list()
    for n in names:
      tmp += list(columns[BEST_NOR][n])
    final_columns[cls] = sorted(set(tmp))
  else:
    final_columns[cls] = sorted(vars)
final_csv = dict()
max_len = max(list(map(lambda x: len(x), final_columns.values())))
for cls, vars in final_columns.items():
  print(cls, len(classifiers[cls]), len(vars))
  ll = max_len - len(vars)
  final_csv[cls] = list(vars) + [''] * ll
ut.save_data(pd.DataFrame(final_csv), home_path + 'final_classifiers.csv', 'csv')

topK_columns = {}
for l in data_labels:
  print(l)
  topK_columns[l] = {}
  for cls, vars in classifiers.items():
    if len(vars) <= 10:
      topK_columns[l][cls] = sorted(vars)
    else:
      topK_columns[l][cls] = columns[BEST_NOR][cls + '_' + l]
  topK_csv = {}
  max_len = max(list(map(lambda x: len(x), topK_columns[l].values())))
  for cls, vars in topK_columns[l].items():
    ll = max_len - len(vars)
    topK_csv[cls] = list(vars) + [''] * ll
  ut.save_data(pd.DataFrame(topK_csv), home_path + f'top10_{l}_classifiers.csv', 'csv')

Metacore&GSEA target genes 26 21
GSEA_R enrichment_all 37 27
GSEA_R enrichment_intersection 11 11
BC Basal 19 18
BC basal-9gene 9 9
BC Luminal 19 32
BC Luminal-16gene 16 15
BC P53 pathway 95 35
BC P53like 19 18
BC P63 pathway 39 26
Basal-squamous_ref.eau-cancer genome 93 37
Luminal_ref.eau-cancer genome 42 28
Luminal-infiltrated_ref.eau-cancer genome 71 30
Luminal-papillary_ref.eau-cancer genome 23 21
Neuronal_ref.eau-cancer genome 94 31
GESA_YES_Genes 27 25
Up_Regulated 40 29
MDA_MVAC
MDA_DDMVAC
Meta_Datasets
AMC


In [11]:
from itertools import permutations
orders = list(range(len(data_labels)))
orders = list(permutations(orders, 2))
orders

[(0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2)]

In [12]:
from scipy.stats import ttest_ind
from itertools import chain

def confirm_by_ttest(arr1, arr2, pvalue=0.05):
  tmp = ttest_ind(arr1, arr2, equal_var=False)
  if tmp.pvalue <= pvalue:
    differ = True
  else:
    differ = False
  return differ, tmp

def modeling_with_various_features(X_tr, y_tr, X_val, y_val, fts_dict, md_mode, Xy_ts_sets=None):
  result = dict()
  result['models'] = dict()
  result['scores_tr_val'] = dict()
  result['ttest_vals'] = dict()
  result['best_classifiers'] = []

  perf = []
  metrics = ['AUC', 'PRAUC', 'R2']
  for k, filtered in fts_dict.items():

    if md_mode == 'logistic':
      tmp = lm.logiReg_model_with_performance([X_tr[filtered], y_tr], 10, class_weight='balanced')
    elif md_mode == 'decision':
      tmp = tm.tree_model_with_performance([X_tr[filtered], y_tr], 3, 3, class_weight='balanced')
    elif md_mode == 'random':
      tmp = tm.random_forest_with_performance([X_tr[filtered], y_tr], 50, 3, 3)
    
    prob_tr = tmp['model'].predict_proba(X_tr[filtered])[:, 1]
    pred_val = tmp['model'].predict(X_val[filtered])
    prob_val = tmp['model'].predict_proba(X_val[filtered])[:, 1]

    result['scores_tr_val'][k] = [prob_tr, prob_val]

    pred_ts_sets = []
    prob_ts_sets = []
    if Xy_ts_sets is not None:
      for X_ts, y_ts in Xy_ts_sets:
        pred_ts_sets.append(tmp['model'].predict(X_ts[filtered]))
        prob_ts_sets.append(tmp['model'].predict_proba(X_ts[filtered])[:, 1])
        result['scores_tr_val'][k] += [prob_ts_sets[-1]]
    
    ### 스코어 검증
    div_tr = []
    div_val = []
    for i in range(2):
      div_tr.append(prob_tr[np.array(y_tr) == i])
      div_val.append(prob_val[np.array(y_val) == i])
    # 1. 동일데이터 내 R vs. NR 차이가 유효한가
    # 2. R 끼리 유사한가
    # 3. NR 끼리 유사한가
    result['ttest_vals'][k] = [confirm_by_ttest(div_tr[0], div_tr[1])[0], confirm_by_ttest(div_val[0], div_val[1])[0], not confirm_by_ttest(div_tr[0], div_val[0])[0], not confirm_by_ttest(div_tr[1], div_val[1])[0]]
    if sum(result['ttest_vals'][k]) == 4:
      result['best_classifiers'].append(k)
    result['models'][k] = tmp

    val = tm.compute_performance(y_val, pred_val, prob_val)
    ts_sets = []
    if Xy_ts_sets is not None:
      for i, (X_ts, y_ts) in enumerate(Xy_ts_sets):
        ts_sets.append(tm.compute_performance(y_ts, pred_ts_sets[i], prob_ts_sets[i]))
    tmp_ = []
    for met in metrics:
      tmp_ += [tmp['performance'][met], val[met]] + list(map(lambda x: x[met], ts_sets))
    perf.append(tmp_)
    print(k, tmp_)

  r_perf = []
  if len(result['best_classifiers']) > 0:
    for c in result['best_classifiers']:
      ii = list(fts_dict.keys()).index(c)
      r_perf.append(perf[ii])
    cols_ = list(map(lambda x: 'ts'+str(x), range(len(ts_sets))))
    cols = list(chain(*map(lambda x: list(map(lambda y: y + '_' + x, ['tr', 'val'] + cols_)), metrics)))
    comp = pd.DataFrame(r_perf, columns=cols, index=result['best_classifiers'])
    print(comp)
  else:
    comp = None
  return result, comp

In [13]:
def validated_models(orders, datasets, best_nor, col_dict, data_labels, over_idx_dict, mode):
  mode_al, mode_cls = mode.split('_')
  models = dict()
  for o1, o2 in orders:
    X_tr, y_tr = datasets[best_nor][o1].iloc[over_idx_dict[data_labels[o1]]], np.array(datasets[best_nor][o1]['response'])[over_idx_dict[data_labels[o1]]]
    X_ts, y_ts = datasets[best_nor][o2], np.array(datasets[best_nor][o2]['response'])
    ts_idxs = sorted({0, 1, 2, 3} - {o1, o2})
    Xy_ts_sets = []
    for i in ts_idxs:
      Xy_ts_sets.append([datasets[best_nor][i], np.array(datasets[best_nor][i]['response'])])
    if mode_cls == 'union':
      cols = col_dict
    elif mode_cls == 'topK':
      cols = col_dict[data_labels[o1]]
    total = modeling_with_various_features(X_tr, y_tr, X_ts, y_ts, cols, mode_al, Xy_ts_sets)
    print(o1, o2, len(total[0]['best_classifiers']), ts_idxs)
    if len(total[0]['best_classifiers']) > 0:
      models['->'.join([data_labels[o1], data_labels[o2]])] = total

  if len(models) > 0:
    ut.save_data(models, home_path + f'{mode_cls}_{mode_al}.pkl', 'pkl')
    tmp = []
    for key in models.keys():
      tt = models[key][1].copy()
      tt.index = list(map(lambda x: key + ',' + x, tt.index))
      tmp.append(tt)
    comp = pd.concat(tmp)
    print(comp)
    return models, comp
  else:
    return None, None

In [ ]:
%%time
log_union, log_union_comp = validated_models(orders, datasets, BEST_NOR, final_columns, data_labels, over_idx_dict, 'logistic_union')
log_topK, log_topK_comp = validated_models(orders, datasets, BEST_NOR, topK_columns, data_labels, over_idx_dict, 'logistic_topK')

In [26]:
ut.save_data(over_idx_dict, home_path + f'over_idx_dict_20211001.pkl', 'pickle')
ut.save_data([log_union, log_union_comp], home_path + f'log_union_info_20211001.pkl', 'pickle')
ut.save_data([log_topK, log_topK_comp], home_path + f'log_topK_info_20211001.pkl', 'pickle')

In [15]:
log_union_comp

,tr_AUC,val_AUC,ts0_AUC,ts1_AUC,tr_PRAUC,val_PRAUC,ts0_PRAUC,ts1_PRAUC,tr_R2,val_R2,ts0_R2,ts1_R2
"Meta_Datasets->MDA_MVAC,BC Luminal",0.741994,0.735294,0.586111,0.511111,0.687387,0.439051,0.654679,0.436664,0.172212,-0.231369,-0.165447,-0.304901
"Meta_Datasets->MDA_MVAC,BC Luminal-16gene",0.670549,0.754902,0.613889,0.602778,0.591809,0.462963,0.616254,0.518405,0.073273,-0.183494,0.079360,-0.045351
"Meta_Datasets->MDA_MVAC,BC P53 pathway",0.730443,0.784314,0.788889,0.494444,0.672328,0.729167,0.798505,0.443455,0.147606,-0.076274,0.261185,-0.243337
"Meta_Datasets->MDA_MVAC,BC P53like",0.672643,0.823529,0.611111,0.638889,0.597033,0.559198,0.687537,0.627207,0.078345,-0.102038,0.013475,0.015077
"Meta_Datasets->MDA_DDMVAC,BC P53 pathway",0.730443,0.788889,0.784314,0.494444,0.672328,0.798505,0.729167,0.443455,0.147606,0.261185,-0.076274,-0.243337
"Meta_Datasets->AMC,BC P63 pathway",0.697290,0.702778,0.539216,0.569444,0.639317,0.721492,0.415741,0.537772,0.111449,0.099566,-0.304868,-0.095558


In [16]:
log_topK_comp

,tr_AUC,val_AUC,ts0_AUC,ts1_AUC,tr_PRAUC,val_PRAUC,ts0_PRAUC,ts1_PRAUC,tr_R2,val_R2,ts0_R2,ts1_R2
"MDA_DDMVAC->MDA_MVAC,Basal-squamous_ref.eau-cancer genome",0.927695,0.852941,0.523006,0.483333,0.901888,0.759259,0.464100,0.404399,0.559197,-0.106881,-0.451207,-0.283844
"MDA_DDMVAC->AMC,Neuronal_ref.eau-cancer genome",0.885772,0.741667,0.647059,0.484076,0.910447,0.692418,0.384790,0.460263,0.455374,0.124183,-0.684563,-0.538099
"Meta_Datasets->MDA_MVAC,BC P53like",0.640381,0.794118,0.616667,0.636111,0.597025,0.504167,0.689357,0.544171,0.058754,-0.151407,0.075770,-0.013564
"Meta_Datasets->MDA_MVAC,Luminal-infiltrated_ref.eau-cancer genome",0.657709,0.764706,0.691667,0.575000,0.602222,0.557738,0.701531,0.496018,0.067064,-0.119595,0.132851,-0.069791
"Meta_Datasets->MDA_DDMVAC,BC Luminal-16gene",0.628914,0.688889,0.637255,0.469444,0.584648,0.671810,0.487121,0.474858,0.044514,0.107458,-0.249453,-0.102667
"Meta_Datasets->MDA_DDMVAC,BC P53 pathway",0.642005,0.672222,0.637255,0.605556,0.593637,0.674811,0.372926,0.502444,0.053189,0.103957,-0.243597,-0.021540
"Meta_Datasets->MDA_DDMVAC,Luminal-infiltrated_ref.eau-cancer genome",0.657709,0.691667,0.764706,0.575000,0.602222,0.701531,0.557738,0.496018,0.067064,0.132851,-0.119595,-0.069791
"Meta_Datasets->AMC,BC P63 pathway",0.648550,0.747222,0.607843,0.541667,0.600304,0.666947,0.452632,0.555401,0.056844,0.081509,-0.235003,-0.019281
"AMC->MDA_DDMVAC,BC Basal",0.882795,0.669444,0.401961,0.507173,0.888702,0.755808,0.256441,0.463110,0.429617,-0.046688,-1.413158,-0.457216
"AMC->MDA_DDMVAC,Luminal_ref.eau-cancer genome",0.942648,0.702778,0.431373,0.517142,0.938742,0.667122,0.269442,0.476784,0.663221,-0.070987,-1.526789,-0.618678


In [ ]:
%%time
dt_union, dt_union_comp = validated_models(orders, datasets, BEST_NOR, final_columns, data_labels, over_idx_dict, 'decision_union')
dt_topK, dt_topK_comp = validated_models(orders, datasets, BEST_NOR, topK_columns, data_labels, over_idx_dict, 'decision_topK')

In [18]:
dt_union_comp

In [19]:
dt_topK_comp

In [ ]:
%%time
rf_union, rf_union_comp = validated_models(orders, datasets, BEST_NOR, final_columns, data_labels, over_idx_dict, 'random_union')
rf_topK, rf_topK_comp = validated_models(orders, datasets, BEST_NOR, topK_columns, data_labels, over_idx_dict, 'random_topK')

In [21]:
rf_union_comp

In [22]:
rf_topK_comp

In [23]:
def draw_box_plots(tag, cls, model, cols, datasets, data_labels):
  tags = tag.split('->')
  tags = [tags[0]] + tags[1].split(',')
  idxs = list(map(lambda x: data_labels.index(x), tags[:2]))
  r_idxs = sorted({0, 1, 2, 3} - set(idxs))
  ll = ['M', 'V', 'T0', 'T1']

  scores = []
  labels = []
  for i, j in enumerate(idxs + r_idxs):
    yy = datasets[j]['response']
    scores.append(model.predict_proba(datasets[j][cols])[:, 1])
    tmp = np.array([f'{i}.{ll[i]}_{data_labels[j]}_NR'] * len(yy))
    tmp[np.array(yy) == 1.0] = f'{i}.{ll[i]}_{data_labels[j]}_R'
    labels.append(tmp)
  
  fe.plot_box(np.concatenate(scores), 'y', np.concatenate(labels), tag + ' + ' + cls)

def draw_all(models, datasets, data_labels):
  for i in models.keys():
    for j in models[i][0]['best_classifiers']:
      draw_box_plots(i, j, models[i][0]['models'][j]['model'], list(models[i][0]['models'][j]['columns']), datasets, data_labels)

In [24]:
draw_all(log_union, datasets[BEST_NOR], data_labels)

In [25]:
draw_all(log_topK, datasets[BEST_NOR], data_labels)

In [ ]:
draw_all(dt_union, datasets[BEST_NOR], data_labels)